In [22]:
import torch.nn as nn
import numpy as np
import torch
import yaml
from torch.autograd import Variable
from torch.utils.data import DataLoader
from txt2image_dataset import Text2ImageDataset
from utils import Utils, Logger
from PIL import Image
import os

### Dataset

In [2]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

config

{'birds_images_path': 'data/cvpr2016_cub/images/',
 'birds_embedding_path': 'data/cub_icml/',
 'birds_text_path': 'data/cvpr2016_cub/cvpr2016_cub/text_c10/',
 'birds_dataset_path': 'data/cvpr2016_cub/text2image/birds.hdf5',
 'val_split_path': 'data/cvpr2016_cub/valclasses.txt',
 'train_split_path': 'data/cvpr2016_cub/trainclasses.txt',
 'test_split_path': 'data/cvpr2016_cub/testclasses.txt',
 'flowers_images_path': 'data/cvpr2016_flowers/images/',
 'flowers_embedding_path': 'data/flowers_icml/',
 'flowers_text_path': 'data/cvpr2016_flowers/text_c10/',
 'flowers_dataset_path': 'data/cvpr2016_flowers/text2image/flowers.hdf5',
 'flowers_val_split_path': 'data/cvpr2016_flowers/valclasses.txt',
 'flowers_train_split_path': 'data/cvpr2016_flowers/trainclasses.txt',
 'flowers_test_split_path': 'data/cvpr2016_flowers/testclasses.txt'}

### Generator

In [3]:
class generator1(nn.Module):
    def __init__(self):
        super(generator1, self).__init__()
        self.image_size = 64
        self.num_channels = 3
        self.noise_dim = 100
        self.ngf = 64

        self.netG = nn.Sequential(
            nn.ConvTranspose2d(self.noise_dim, self.ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(self.ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (num_channels) x 64 x 64
            )

    def forward(self, z):
        return self.netG(z)

In [5]:
generator1()

generator1(
  (netG): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [4]:
class discriminator1(nn.Module):
    def __init__(self):
        super(discriminator1, self).__init__()
        self.image_size = 64
        self.num_channels = 3
        self.ndf = 64

        self.netD_1 = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(self.ndf * 8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.netD_2 = nn.Sequential(
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
            )

    def forward(self, inp):
        x_intermediate = self.netD_1(inp)
        output = self.netD_2(x_intermediate)
        return output.view(-1, 1).squeeze(1), x_intermediate

In [6]:
discriminator1()

discriminator1(
  (netD_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (netD_2): Sequential(
    (0): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): Sigmoid()
  )
)

In [53]:
noise_dim = 100
batch_size = 64
num_workers = 1
lr = 0.0002
beta1 = 0.5
num_epochs = 1
l1_coef = 50
l2_coef = 200

vis_screen = 'gan'
checkpoints_path = 'checkpoints'
save_path = 'results'
logger = Logger(vis_screen)

Setting up a new session...


In [54]:
criterion = nn.BCELoss()
l2_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

In [55]:
# generator
generator = generator1().cuda()
generator.apply(Utils.weights_init)

# discriminator
discriminator = discriminator1().cuda()
discriminator.apply(Utils.weights_init)

optimD = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
optimG = torch.optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))


In [56]:
def train(ds='flowers'):

    if ds == 'birds':
        dataset = Text2ImageDataset(config['birds_dataset_path'], split=0)
    if ds == 'flowers':
        dataset = Text2ImageDataset(config['flowers_dataset_path'], split=0)

    data_loader = DataLoader(dataset,
                            batch_size=batch_size,
                            shuffle=True)

    iteration = 0
    
    print("num iterations", len(data_loader), 'batch_size', batch_size)
    
    for epoch in range(num_epochs):
        for sample in data_loader:
            iteration += 1
            right_images = sample['right_images']
            # print(f"right_images shape {right_images.shape}")

            right_images = Variable(right_images.float()).cuda()

            real_labels = torch.ones(right_images.size(0))
            fake_labels = torch.zeros(right_images.size(0))

            smoothed_real_labels = torch.FloatTensor(Utils.smooth_label(real_labels.numpy(), -0.1))

            real_labels = Variable(real_labels).cuda()
            smoothed_real_labels = Variable(smoothed_real_labels).cuda()
            fake_labels = Variable(fake_labels).cuda()

            # Train the discriminator
            discriminator.zero_grad()
            outputs, activation_real = discriminator(right_images)
            # print(f"outputs shape {outputs.shape}, activation shape {activation_real.shape}")
            real_loss = criterion(outputs, smoothed_real_labels)
            real_score = outputs

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)
            # print(f"noise shape {noise.shape}")
            fake_images = generator(noise)
            # print(f"fake_images shape {fake_images.shape}")
            outputs, _ = discriminator(fake_images)
            fake_loss = criterion(outputs, fake_labels)
            fake_score = outputs

            d_loss = real_loss + fake_loss

            d_loss.backward()
            optimD.step()

            # Train the generator
            generator.zero_grad()

            noise = Variable(torch.randn(right_images.size(0), 100)).cuda()
            noise = noise.view(noise.size(0), 100, 1, 1)

            fake_images = generator(noise)

            outputs, activation_fake = discriminator(fake_images)
            _, activation_real = discriminator(right_images)

            activation_fake = torch.mean(activation_fake, 0)
            activation_real = torch.mean(activation_real, 0)

            g_loss = criterion(outputs, real_labels) \
                     + l2_coef * l2_loss(activation_fake, activation_real.detach()) \
                     + l1_coef * l1_loss(fake_images, right_images)

            g_loss.backward()
            optimG.step()
            
            if iteration % 5 == 0:
                logger.log_iteration_gan(epoch, d_loss, g_loss, real_score, fake_score)
                logger.draw(right_images, fake_images)

        logger.plot_epoch_w_scores(iteration)
        
        if epoch % 50 == 0:
            Utils.save_checkpoint(discriminator, generator, checkpoints_path, epoch)



In [57]:
train()

num iterations 460 batch_size 64
Epoch: 0, d_loss= 1.243321, g_loss= 38.609428, D(X)= 0.805490, D(G(X))= 0.435543
Epoch: 0, d_loss= 1.055493, g_loss= 33.253956, D(X)= 0.530675, D(G(X))= 0.087966
Epoch: 0, d_loss= 1.209785, g_loss= 32.987419, D(X)= 0.473177, D(G(X))= 0.051496
Epoch: 0, d_loss= 0.763523, g_loss= 35.347191, D(X)= 0.650580, D(G(X))= 0.033794
Epoch: 0, d_loss= 0.583605, g_loss= 37.259064, D(X)= 0.817344, D(G(X))= 0.050077
Epoch: 0, d_loss= 0.513030, g_loss= 36.827385, D(X)= 0.781284, D(G(X))= 0.038840
Epoch: 0, d_loss= 0.447854, g_loss= 37.943108, D(X)= 0.872314, D(G(X))= 0.055344
Epoch: 0, d_loss= 0.478810, g_loss= 37.961006, D(X)= 0.923322, D(G(X))= 0.071002
Epoch: 0, d_loss= 0.548646, g_loss= 36.413177, D(X)= 0.882782, D(G(X))= 0.127544
Epoch: 0, d_loss= 1.360720, g_loss= 38.055511, D(X)= 0.957994, D(G(X))= 0.497462
Epoch: 0, d_loss= 2.282389, g_loss= 35.352516, D(X)= 0.926820, D(G(X))= 0.741529
Epoch: 0, d_loss= 0.654803, g_loss= 30.798615, D(X)= 0.670942, D(G(X))= 0.09

TypeError: save_checkpoint() missing 1 required positional argument: 'epoch'

[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10054] An existing connection was forcibly closed by the remote host
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine activel

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

...

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively

[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively refused it
[WinError 10061] No connection could be made because the target machine actively